## Read in Smartfin data taken only from surf sessions around Scripps pier. 

In [10]:
#Imports 
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import re
import random
import math

print("Done.")

Done.


### Smartfin is collecting data at rate of 5 Hz --> every 0.2 seconds. (Note: Acceleration is in m/s^2, Gyroscope is in degrees/sec). 

In [2]:
# Read in the combined file: 
scripps_df = pd.read_csv('Scripps_Final_Combined.csv')
scripps_df = scripps_df.drop(columns=['Unnamed: 0'])
scripps_df[:10]

,ID,UTC,Time,IMU A1,IMU A2,IMU A3,IMU G1,IMU G2,IMU G3,IMU M1,IMU M2,IMU M3,Date,Hs(m),Tp(s),Dp(deg)
0,google_106807627751629413403_0006667E228C_1709...,2017-09-09T18:02:11.2810+00:00,18:02,0.000000,11.070788,0.134075,46.829268,15.853659,15.487805,-37.0,-199.0,517.0,2017-09-09,0.841248,5.88,284.0
1,google_106807627751629413403_0006667E228C_1709...,2017-09-09T18:02:11.4800+00:00,18:02,-0.325611,11.185710,-0.459687,8.902439,-2.439024,12.073171,-33.0,-179.0,533.0,2017-09-09,0.841248,5.88,284.0
2,google_106807627751629413403_0006667E228C_1709...,2017-09-09T18:02:11.7310+00:00,18:02,0.000000,10.419566,1.800440,-16.341463,1.463415,14.268293,-46.0,-188.0,528.0,2017-09-09,0.841248,5.88,284.0
3,google_106807627751629413403_0006667E228C_1709...,2017-09-09T18:02:11.9810+00:00,18:02,0.823605,10.860099,-0.114922,-5.731707,17.439024,21.707317,-64.0,-186.0,522.0,2017-09-09,0.841248,5.88,284.0
4,google_106807627751629413403_0006667E228C_1709...,2017-09-09T18:02:12.1800+00:00,18:02,0.670376,11.492168,-0.651223,-28.658537,1.219512,21.341463,-91.0,-175.0,493.0,2017-09-09,0.841248,5.88,284.0
5,google_106807627751629413403_0006667E228C_1709...,2017-09-09T18:02:12.4310+00:00,18:02,0.517148,10.649409,1.877054,-3.536585,3.292683,8.292683,-100.0,-182.0,486.0,2017-09-09,0.841248,5.88,284.0
6,google_106807627751629413403_0006667E228C_1709...,2017-09-09T18:02:12.6310+00:00,18:02,0.114922,10.189722,0.995988,12.682927,17.682927,-5.487805,-102.0,-180.0,474.0,2017-09-09,0.841248,5.88,284.0
7,google_106807627751629413403_0006667E228C_1709...,2017-09-09T18:02:12.8310+00:00,18:02,0.172383,9.289502,0.383072,16.463415,13.780488,-25.609756,-97.0,-183.0,487.0,2017-09-09,0.841248,5.88,284.0
8,google_106807627751629413403_0006667E228C_1709...,2017-09-09T18:02:13.0800+00:00,18:02,0.517148,6.876147,0.919373,20.975610,9.878049,-42.317073,-53.0,-189.0,497.0,2017-09-09,0.841248,5.88,284.0
9,google_106807627751629413403_0006667E228C_1709...,2017-09-09T18:02:13.3310+00:00,18:02,-0.095768,7.067683,-0.248997,28.658537,2.804878,-17.439024,-8.0,-182.0,532.0,2017-09-09,0.841248,5.88,284.0


In [3]:
# Drop the UTC, Time, and Date columns
scripps_df = scripps_df.drop(columns=['UTC', 'Time', 'Date'])
scripps_df[:10]

,ID,IMU A1,IMU A2,IMU A3,IMU G1,IMU G2,IMU G3,IMU M1,IMU M2,IMU M3,Hs(m),Tp(s),Dp(deg)
0,google_106807627751629413403_0006667E228C_1709...,0.000000,11.070788,0.134075,46.829268,15.853659,15.487805,-37.0,-199.0,517.0,0.841248,5.88,284.0
1,google_106807627751629413403_0006667E228C_1709...,-0.325611,11.185710,-0.459687,8.902439,-2.439024,12.073171,-33.0,-179.0,533.0,0.841248,5.88,284.0
2,google_106807627751629413403_0006667E228C_1709...,0.000000,10.419566,1.800440,-16.341463,1.463415,14.268293,-46.0,-188.0,528.0,0.841248,5.88,284.0
3,google_106807627751629413403_0006667E228C_1709...,0.823605,10.860099,-0.114922,-5.731707,17.439024,21.707317,-64.0,-186.0,522.0,0.841248,5.88,284.0
4,google_106807627751629413403_0006667E228C_1709...,0.670376,11.492168,-0.651223,-28.658537,1.219512,21.341463,-91.0,-175.0,493.0,0.841248,5.88,284.0
5,google_106807627751629413403_0006667E228C_1709...,0.517148,10.649409,1.877054,-3.536585,3.292683,8.292683,-100.0,-182.0,486.0,0.841248,5.88,284.0
6,google_106807627751629413403_0006667E228C_1709...,0.114922,10.189722,0.995988,12.682927,17.682927,-5.487805,-102.0,-180.0,474.0,0.841248,5.88,284.0
7,google_106807627751629413403_0006667E228C_1709...,0.172383,9.289502,0.383072,16.463415,13.780488,-25.609756,-97.0,-183.0,487.0,0.841248,5.88,284.0
8,google_106807627751629413403_0006667E228C_1709...,0.517148,6.876147,0.919373,20.975610,9.878049,-42.317073,-53.0,-189.0,497.0,0.841248,5.88,284.0
9,google_106807627751629413403_0006667E228C_1709...,-0.095768,7.067683,-0.248997,28.658537,2.804878,-17.439024,-8.0,-182.0,532.0,0.841248,5.88,284.0


### Take only the mean and standard deviation as input features, sampled over an interval x.

In [4]:
# Only get the columns with 'IMU' in their title. 
imu_cols = []
for c in scripps_df.columns: 
    if 'IMU' in c:
        imu_cols.append(c)
print(imu_cols)

['IMU A1', 'IMU A2', 'IMU A3', 'IMU G1', 'IMU G2', 'IMU G3', 'IMU M1', 'IMU M2', 'IMU M3']


In [5]:
# Create the column names for the two statistical input feature column names that I want. 

mean_cols = []
std_cols = []
for c in imu_cols: 
    mean_cols.append(c + ' Mean')
    std_cols.append(c + ' STD')
    
print(mean_cols)
print(std_cols)

['IMU A1 Mean', 'IMU A2 Mean', 'IMU A3 Mean', 'IMU G1 Mean', 'IMU G2 Mean', 'IMU G3 Mean', 'IMU M1 Mean', 'IMU M2 Mean', 'IMU M3 Mean']
['IMU A1 STD', 'IMU A2 STD', 'IMU A3 STD', 'IMU G1 STD', 'IMU G2 STD', 'IMU G3 STD', 'IMU M1 STD', 'IMU M2 STD', 'IMU M3 STD']


In [16]:
# Create a new dataframe that is sampled at intervals based on the old dataframe and includes 
# statistical properties (mean, std) of those sampled intervals as new column features.

scripps_mean_df = pd.DataFrame(columns=mean_cols)
scripps_std_df = pd.DataFrame(columns=std_cols)

progress = 0
interval = 5  # interval = 5 corresponds to 5 * 0.2 = every 1 second
x = interval
for i in range(0, math.floor(len(scripps_df)/x)):
    mini_df = scripps_df[i*x:x*(i+1)]
    #print("Printing mini df:")
    #print(mini_df) # problem where i goes from 0:, 1: (should be 0:, 5:)
    
    # Check to make sure that the ID is the same (taken from same surf session):
    first_id = mini_df['ID'].iloc[0]
    last_id = mini_df['ID'].iloc[-1]
    if first_id == last_id: 
        mean_list = []
        std_list = []
        for c in mini_df.columns:
            if 'IMU' in c:
                mean_list.append(np.mean(mini_df[c]))
                std_list.append(np.std(mini_df[c]))
        scripps_mean_df.loc[len(scripps_mean_df)] = mean_list
        progress += 1
        if progress % 1000 == 0:
            print(progress)

print(scripps_mean_df)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

## To do: preserve the ID field in the above computation so we can repeat the above for different intervals. 

In [14]:
print(len(scripps_df))

1538168


In [8]:
# Choose what interval to iterate over in minutes (sampling frequency for this was 5 Hz)
def stat_intervals(interval_mins, samp_freq): 
    count = 0
    counter = samp_freq*60*interval_mins
    
    old_id = ''
    
    for index, row in scripps_df.iterrows():
        print(index, row)
        ID = row['id']
        #if ID != old_id and count != 0: 
        #   mean = 
    

In [ ]:
# Compute the mean and std of each row
# Function with parameters of 1, 5, 10, 20, 30, entire surf session intervals

In [ ]:
# Machine learning models